## Ingesting PDF

In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
import pdf2image
import numpy as np
import cv2

In [3]:
local_path = "Newport A student.pdf"


# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [4]:
# Preview first page
data[0].page_content

'Contents\n\nCover Page Title Page Introduction\n\nPart 1. Study Basics\n\nStep 1 Manage Your Time in Five Minutes a\n\nDay\n\nStep 2 Declare War on Procrastination Step 3 Choose When, Where, and How Long\n\nPart One Cheat Sheet\n\nPart 2. Quizzes and Exams\n\nStep 1 Take Smart Notes Step 2 Demote Your Assignments Step 3 Marshal Your Resources Step 4 Conquer the Material Step 5 Invest in “Academic Disaster Insurance” Step 6 Provide “A+” Answers\n\nThe Plan in Action\n\nPart Two Cheat Sheet\n\nPart 3. Essays and Papers\n\nStep 1 Target a Titillating Topic Step 2 Conduct a Thesis-Hunting Expedition Step 3 Seek a Second Opinion Step 4 Research like a Machine Step 5 Craft a Powerful Story Step 6 Consult Your Expert Panel Step 7 Write Without the Agony Step 8 Fix, Don’t Fixate\n\nThe Plan in Action\n\nPart Three Cheat Sheet\n\nConclusion Acknowledgments Getting in is just the beginning Copyright Page\n\nIntroduction\n\n“My friends always wondered why I was never in the library, but instead 

## Vector Embeddings

In [5]:
!ollama pull nomic-embed-text

pulling manifest â ‹ pulling manifest 
Error: pull model manifest: Get "https://registry.ollama.ai/v2/library/nomic-embed-text/manifests/latest": dial tcp: lookup registry.ollama.ai: no such host


In [6]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED   
nomic-embed-text:latest	0a109f422b47	274 MB	2 days ago	
llama2:7b              	78e26419b446	3.8 GB	3 days ago	


In [7]:
#%pip install --q chromadb
#%pip install --q langchain-text-splitters

In [8]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [9]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [10]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████| 42/42 [15:09<00:00, 21.65s/it]


## Retrieval

In [11]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [12]:
# LLM from Ollama
local_model = "llama2:7b"
llm = ChatOllama(model=local_model)

In [13]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [14]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input(""))


In [ ]:
chain.invoke("What are the best techniques for a student to be an A student ?")

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()